In [1]:
!pip install pandas numpy scikit-learn joblib seaborn matplotlib

In [1]:
!pip install pandas numpy requests matplotlib seaborn


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os

def load_and_preprocess_data(file_path):
    """
    Load dataset from a CSV file and handle missing values.

    Parameters:
        file_path (str): Path to the CSV file.

    Returns:
        DataFrame: Preprocessed dataset.
        List: List of categorical column names.
    """
    data = pd.read_csv(file_path, low_memory=False)

    # Check for mixed types and fix them in categorical columns
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical columns
            # Convert mixed types to strings or handle them
            data[col] = data[col].astype(str).fillna('missing')

    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns

    # Fill missing values
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())
    data[categorical_cols] = data[categorical_cols].fillna(data[categorical_cols].mode().iloc[0])

    return data, categorical_cols

def encode_categorical_columns(data, categorical_cols):
    """
    Encode categorical columns using LabelEncoder.

    Parameters:
        data (DataFrame): Dataset containing categorical columns.
        categorical_cols (List): List of categorical column names.

    Returns:
        DataFrame: Dataset with encoded categorical columns.
    """
    for col in categorical_cols:
        data[col] = LabelEncoder().fit_transform(data[col])
    return data

def train_and_evaluate_model(data):
    """
    Train a Random Forest Regressor and evaluate the model.

    Parameters:
        data (DataFrame): Dataset with features and target.

    Returns:
        RandomForestRegressor: Trained model.
    """
    X = data.drop(columns=["Risk_Factor"])
    y = data["Risk_Factor"]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Standardize and apply PCA
    scaler = StandardScaler()
    pca = PCA(n_components=0.95)

    X_train = pca.fit_transform(scaler.fit_transform(X_train))
    X_test = pca.transform(scaler.transform(X_test))

    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate model
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Model RMSE: {rmse}")
    return model

def generate_low_stock_alerts(data, stock_column='Economic_Factor', factor=0.3):
    """
    Simulate stock levels and identify low stock items.

    Parameters:
        data (DataFrame): Dataset containing stock information.
        stock_column (str): Column used to simulate stock levels.
        factor (float): Threshold factor to identify low stock items.

    Returns:
        DataFrame: Filtered dataset with low stock items.
    """
    # Simulate stock levels
    data['Simulated_Stock_Level'] = data[stock_column] * 100

    # Set threshold for low stock alerts
    threshold = data['Simulated_Stock_Level'].mean() * factor

    # Filter low stock items
    low_stock_items = data[data['Simulated_Stock_Level'] < threshold]
    return low_stock_items

def save_to_csv(data, filename):
    """
    Save DataFrame to CSV file.

    Parameters:
        data (DataFrame): Dataset to be saved.
        filename (str): Path to the output CSV file.
    """
    data.to_csv(filename, index=False)
    print(f"Risk alerts saved to {filename}")

def send_email_with_attachment(sender_email, sender_password, recipient_email, subject, body, attachment_path):
    """
    Send an email with an attachment.

    Parameters:
        sender_email (str): Sender's email address.
        sender_password (str): Sender's email password.
        recipient_email (str): Recipient's email address.
        subject (str): Email subject.
        body (str): Email body.
        attachment_path (str): Path to the attachment file.
    """
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # Attach the file
    with open(attachment_path, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename={os.path.basename(attachment_path)}')
        msg.attach(part)

    # Send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)

    print(f"Email sent to {recipient_email}")

# Main execution
file_path = '/content/drive/MyDrive/data_risk.csv'  # Update this path to your dataset file
output_file = 'low_stock_alerts.csv'

# Load and preprocess data
data, categorical_cols = load_and_preprocess_data(file_path)
data = encode_categorical_columns(data, categorical_cols)

# Train model (optional: if you're only interested in risk alerts, you can skip this step)
model = train_and_evaluate_model(data)

# Generate low stock alerts
low_stock_items = generate_low_stock_alerts(data)

# Save low stock alerts to CSV
save_to_csv(low_stock_items, output_file)

# Email configuration
sender_email = 'manjushag2006@gmail.com'
sender_password = 'dqbw lywd hcbv cliz'

# Prompt the user for the recipient's email
recipient_email = input("Enter the recipient's email address: ")

subject = 'Low Stock Alerts'
body = 'Please find attached the low stock alerts CSV file.'

# Send the email
send_email_with_attachment(sender_email, sender_password, recipient_email, subject, body, output_file)


Model RMSE: 0.0020645395303264475
Risk alerts saved to low_stock_alerts.csv
Enter the recipient's email address: gvnsmanju@gmail.com
Email sent to gvnsmanju@gmail.com


In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Load and preprocess data
def load_and_preprocess_data(file_path):
    try:
        data = pd.read_csv(file_path, low_memory=False)
    except FileNotFoundError:
        print("File not found. Please check the file path.")
        return None, None

    # Convert categorical columns to string and fill missing values
    for col in data.columns:
        if data[col].dtype == 'object':
            data[col] = data[col].astype(str).fillna('missing')

    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns

    # Fill missing values
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())
    data[categorical_cols] = data[categorical_cols].fillna(data[categorical_cols].mode().iloc[0])

    return data, categorical_cols

# Encode categorical columns
def encode_categorical_columns(data, categorical_cols):
    for col in categorical_cols:
        data[col] = LabelEncoder().fit_transform(data[col])
    return data

# Generate low stock alerts based on a threshold
def generate_low_stock_alerts(data, stock_column='Economic_Factor', factor=0.3):
    # Simulate stock levels
    data['Simulated_Stock_Level'] = data[stock_column] * 100
    threshold = data['Simulated_Stock_Level'].mean() * factor
    low_stock_items = data[data['Simulated_Stock_Level'] < threshold]

    # Filter out unnecessary columns for the final alert file
    low_stock_items_alerts = low_stock_items[['id', 'name', 'price(₹)', 'Is_discontinued', 'manufacturer_name',
                                              'type', 'pack_size_label', 'short_composition1', 'short_composition2',
                                              'Supplier_Performance', 'Economic_Factor', 'Transport_Status', 'Risk_Factor']]

    return low_stock_items_alerts

# Generate suggestions based on row data
def generate_suggestions(row):
    suggestions = []
    if row.get('Is_discontinued', False):
        suggestions.append("⚠️ Product is discontinued. Consider phasing out stock or finding alternatives.")

    if row.get('Supplier_Performance') is not None:
        supplier_score = 1 - row['Supplier_Performance'] if isinstance(row['Supplier_Performance'], float) else 0
        if supplier_score > 0.5:
            suggestions.append("⚠️ Low supplier performance. Consider switching suppliers or improving relationships.")

    if row.get('Economic_Factor') and row['Economic_Factor'] < 0.5:
        suggestions.append("⚠️ Economic factor shows risk. Adjust inventory levels and plan for delays.")

    transport_status = str(row.get('Transport_Status', '')).lower()
    if not transport_status.startswith("on time"):
        suggestions.append("⚠️ Transport status is delayed. Ensure proper logistics tracking.")

    risk_score = row.get('Risk_Factor', 0)
    if risk_score > 0.7:
        suggestions.append("🔴 High risk. Consider reducing production or increasing prices to manage demand.")
    elif risk_score > 0.4:
        suggestions.append("🟡 Moderate risk. Monitor market conditions and supply chain regularly.")
    else:
        suggestions.append("🟢 Low risk. Current operations are stable.")

    sentiment_score = 1.0 - risk_score
    return {'risk_score': risk_score, 'sentiment_score': sentiment_score, 'suggestions': suggestions}

# Generate the summary message
def generate_alert_message(medicine_data):
    medicine_price = medicine_data['price(₹)'].sum()  # Sum of all medicine prices
    risk_score = medicine_data['Risk_Factor'].iloc[0]  # Risk score for the first row (or aggregate logic)
    sentiment_score = 1.0 - risk_score  # Calculate sentiment score

    message = f"Medicine: {medicine_price}\n"
    message += f"Risk Score: {risk_score:.3f}\n"
    message += f"Sentiment Score: {sentiment_score:.3f}\n"
    message += "Suggestions:\n"

    # Get suggestions based on the row
    row_suggestions = generate_suggestions(medicine_data.iloc[0])
    message += '\n'.join(row_suggestions['suggestions'])

    return message

# Save alerts to CSV file
def save_to_csv(data, filename):
    data.to_csv(filename, index=False)
    print(f"Alerts saved to {filename}")

# Send email with low stock alerts and suggestions
def send_email_with_attachment(smtp_server, port, sender_email, sender_password, recipient_email, subject, body, attachment_file):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Attach the file
    with open(attachment_file, 'rb') as file:
        attach = MIMEText(file.read(), 'base64', 'utf-8')
        attach.add_header('Content-Disposition', 'attachment', filename=attachment_file)
        msg.attach(attach)

    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())

    print(f"Email sent to {recipient_email}")

# Main execution
file_path = '/content/drive/MyDrive/data_risk.csv'  # Path to your dataset
output_file = '/content/drive/MyDrive/low_stock_alerts.csv'  # Path for saving the output CSV

# Load and preprocess data
data, categorical_cols = load_and_preprocess_data(file_path)

if data is None:
    print("Error in loading data. Exiting.")
    exit()

data = encode_categorical_columns(data, categorical_cols)

# Generate low stock alerts
low_stock_alerts = generate_low_stock_alerts(data)

# Save low stock alerts to CSV
save_to_csv(low_stock_alerts, output_file)

# Generate message for email
alert_message = generate_alert_message(low_stock_alerts)

# Email configuration
sender_email = 'manjushag2006@gmail.com'
sender_password = 'dqbw lywd hcbv cliz'  # App-specific password for Gmail

# Get recipient email from user
recipient_email = input("Enter the recipient email: ").strip()

# Send the email with the low stock alerts
send_email_with_attachment(
    'smtp.gmail.com', 587, sender_email, sender_password, recipient_email,
    'Medicine Risk and Low Stock Alerts', alert_message, output_file
)


Alerts saved to /content/drive/MyDrive/low_stock_alerts.csv
Enter the recipient email: gvnsmanju@gmail.com
Email sent to gvnsmanju@gmail.com


In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os

# Load and preprocess data
def load_and_preprocess_data(file_path):
    try:
        data = pd.read_csv(file_path, low_memory=False)
    except FileNotFoundError:
        print("File not found. Please check the file path.")
        return None, None

    # Convert all object columns to string and fill missing values with 'missing'
    for col in data.columns:
        if data[col].dtype == 'object':
            data[col] = data[col].astype(str).fillna('missing')

    # Fill missing numeric columns with mean and categorical columns with mode
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    categorical_cols = data.select_dtypes(exclude=[np.number]).columns
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())
    data[categorical_cols] = data[categorical_cols].fillna(data[categorical_cols].mode().iloc[0])

    return data, categorical_cols

def encode_categorical_columns(data, categorical_cols):
    for col in categorical_cols:
        data[col] = LabelEncoder().fit_transform(data[col])
    return data

# Generate suggestions based on the risk factors and other parameters
def generate_suggestions(row):
    suggestions = []

    # Check if product is discontinued
    if row.get('Is_discontinued', False):
        suggestions.append("⚠️ Product is discontinued. Consider phasing out stock or finding alternatives.")

    # Check supplier performance and provide suggestions
    if row.get('Supplier_Performance') is not None:
        supplier_score = 1 - row['Supplier_Performance'] if isinstance(row['Supplier_Performance'], float) else 0
        if supplier_score > 0.5:
            suggestions.append("⚠️ Low supplier performance. Consider switching suppliers or improving relationships.")

    # Check economic factors for any risks
    if row.get('Economic_Factor') and row['Economic_Factor'] < 0.5:
        suggestions.append("⚠️ Economic factor shows risk. Adjust inventory levels and plan for delays.")

    # Check transport status
    transport_status = str(row.get('Transport_Status', '')).lower()
    if not transport_status.startswith("on time"):
        suggestions.append("⚠️ Transport status is delayed. Ensure proper logistics tracking.")

    # Risk factor analysis
    risk_score = row.get('Risk_Factor', 0)
    if risk_score > 0.7:
        suggestions.append("🔴 High risk. Consider reducing production or increasing prices to manage demand.")
    elif risk_score > 0.4:
        suggestions.append("🟡 Moderate risk. Monitor market conditions and supply chain regularly.")
    else:
        suggestions.append("🟢 Low risk. Current operations are stable.")

    # Sentiment score is inversely proportional to risk
    sentiment_score = 1.0 - risk_score
    return {'risk_score': risk_score, 'sentiment_score': sentiment_score, 'suggestions': suggestions}

# Send email using SMTP
def send_email(smtp_server, port, sender_email, sender_password, recipient_email, subject, body, attachment_file, file_path):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    # Attach the CSV file
    with open(attachment_file, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename={os.path.basename(attachment_file)}")
        msg.attach(part)

    # Add file path to email body
    body += f"\n\nThe file containing suggestions can be found at: {file_path}"

    # Connect to SMTP server and send the email
    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls()  # Enable encryption for security
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())

    print(f"Email sent to {recipient_email} with file path included")

# Main execution flow
file_path = '/content/drive/MyDrive/data_risk.csv'  # Path to your dataset

# Load and preprocess the data
data, categorical_cols = load_and_preprocess_data(file_path)

if data is None:
    print("Error in loading data. Exiting.")
    exit()

# Encode categorical columns
data = encode_categorical_columns(data, categorical_cols)

# Get runtime inputs for email details
sender_email = 'manjushag2006@gmail.com'  # Gmail address
sender_password = 'dqbw lywd hcbv cliz'  # App-specific password for Gmail
recipient_email = input("Enter the recipient email: ").strip()  # Recipient email for email notifications

# Process each row and send notifications
medicine_name = input("Enter medicine name: ").strip()  # Get user input for medicine search

medicine_data = data[data['name'].astype(str).str.contains(medicine_name, case=False)]

if medicine_data.empty:
    print("Medicine not found.")
else:
    # Generate suggestions and create a list to save to CSV
    suggestions_list = []

    for _, row in medicine_data.iterrows():
        suggestion = generate_suggestions(row)
        email_subject = f"Medicine Alert: {row['name']}"
        email_body = (
            f"Medicine: {row['name']}\n"
            f"Risk Score: {suggestion['risk_score']}\n"
            f"Sentiment Score: {suggestion['sentiment_score']}\n"
            f"Suggestions:\n" + "\n".join(suggestion['suggestions'])
        )

        # Save the suggestion to CSV
        suggestions_list.append({
            "Medicine": row['name'],
            "Risk Score": suggestion['risk_score'],
            "Sentiment Score": suggestion['sentiment_score'],
            "Suggestions": "; ".join(suggestion['suggestions'])
        })

    # Save all suggestions to a CSV file
    suggestions_df = pd.DataFrame(suggestions_list)
    attachment_file = '/content/drive/MyDrive/low_stock_alerts.csv'
    suggestions_df.to_csv(attachment_file, index=False)

    # Get the full file path of the CSV
    file_path_to_send = os.path.abspath(attachment_file)

    # Send email notification
    send_email('smtp.gmail.com', 587, sender_email, sender_password, recipient_email, email_subject, email_body, attachment_file, file_path_to_send)


Enter the recipient email: gvnsmanju@gmail.com
Enter medicine name: 18143
Email sent to gvnsmanju@gmail.com with file path included
